In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2022-12-15 08:49:53--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  87.2MB/s    in 0.3s    

2022-12-15 08:49:53 (87.2 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [4]:
# add your code here - consider creating a new cell for each section of code

In [5]:
users_rating = (df_ratings.groupby(by = ['user'])['rating'].count().reset_index().rename(columns = {'rating': 'userRating'}))[['user', 'userRating']]
significant_users = users_rating.query('userRating > 200').user.tolist()

In [6]:
# merge rating and catalog by isbn
book_df = pd.merge(df_ratings,df_books,on='isbn')

books_rating = (book_df.
     groupby(by = ['title'])['rating'].
     count().
     reset_index().
     rename(columns = {'rating': 'bookRating'})
     [['title', 'bookRating']]
    )

In [7]:
books_ratings = book_df.merge(books_rating, on='title')

In [8]:
books_df_after_rating_query = books_ratings.query('bookRating > 100')

In [9]:
books_to_analyze = books_df_after_rating_query[books_df_after_rating_query['user'].isin(significant_users)]

In [10]:
training_set = books_to_analyze.pivot_table(index='title',columns='user',values='rating').fillna(0)

In [18]:
model_nn = NearestNeighbors(metric='cosine')
model_nn.fit(training_set.values)

NearestNeighbors(metric='cosine')

In [23]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):

  analyzed_book = training_set.loc[book]
  distance, idx = model_nn.kneighbors([analyzed_book.values], n_neighbors = 5)

  recommended_books = pd.DataFrame({
      'book_title'   : training_set.iloc[idx[0]].index.values,
      'distance': distance[0]}).sort_values(by='distance', ascending=False).values

  return [book, recommended_books]

In [24]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", array([['The Weight of Water', 0.7708583474159241],
       ['I Know This Much Is True', 0.7529292702674866],
       ['The Lovely Bones: A Novel', 0.7230184078216553],
       ['Blue Diary', 0.718287467956543],
       ["Where the Heart Is (Oprah's Book Club (Paperback))", 0.0]],
      dtype=object)]
You passed the challenge! 🎉🎉🎉🎉🎉
